In [ ]:
!pip install duckdb

In [ ]:
%matplotlib inline

# adding project dirs to path so code may be referenced from the notebook
import sys
sys.path.insert(0, '..')

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import config
import utils

# Query some forcast data from parquet files
import importlib
import queries
importlib.reload(queries)

In [ ]:
# load huc10
%time
basins_gdf = utils.parquet_to_gdf(config.HUC10_PARQUET_FILEPATH)

# Filter to CA
ca_gdf = basins_gdf.loc[basins_gdf["huc10"].str.startswith("18")]
ca_gdf.plot()

In [ ]:
# load usgs gages
%time
usgs_gages = utils.get_usgs_gages()

In [ ]:
ca_usgs_gages = gpd.sjoin(usgs_gages, ca_gdf,how='inner', predicate='within')

In [ ]:
ca_usgs_gages.plot(markersize=1.5)

In [ ]:
ca_nwm_fids = list(ca_usgs_gages["nwm_feature_id"])

In [ ]:
query = queries.calculate_nwm_feature_metrics(
    config.MEDIUM_RANGE_PARQUET,
    config.USGS_PARQUET,
    group_by=["nwm_feature_id"],
    order_by=["observed_average"],
    filters=[
        {
            "column": "reference_time",
            "operator": "=",
            "value": "2023-01-03 12:00:00"
        }
    ]
)
print(query)
df = duckdb.query(query).to_df()
df

In [ ]:
# Join query to basins
gdf_map = ca_usgs_gages.merge(df, left_on="nwm_feature_id", right_on="nwm_feature_id")

gdf_map.plot("observed_average", legend=True, markersize=1.5)